In [1]:
# 기본 라이브러리
import os
import sys
import numpy as np
import pandas as pd
import torch
import re
from tqdm import tqdm

# 데이터 처리 관련
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

# Hugging Face Transformers
from transformers import (
    BertConfig, AutoTokenizer, BertForSequenceClassification, AdamW
)

# 평가 관련 (F1 Score, Classification Report)
from seqeval.metrics import f1_score, classification_report

# Config 불러오기
import config

# 라벨 변환 관련 모듈
import label

# 데이터셋 관련 모듈
import Dataset

sys.path.append('/home/lhch9550/공모전/KPF-BERT-CLS')

# 데이터 로딩
data = pd.read_csv('/home/lhch9550/공모전/KPF-BERT-CLS/processed_data.csv')  
print("Data loaded!")

# 텍스트 클리닝 함수 정의
def clean_text(text):
    text = re.sub("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z-.]+)", "", text)  # 이메일 제거
    text = re.sub("[\r\n\xa0]", "", text)  # 줄 바꿈 및 특수문자 제거
    text = re.sub("(\.\s+[ㄱ-ㅎ가-힣]+\s[기]+[자]+)", "", text)  # 특정한 문장 패턴 제거
    text = re.sub("[^\w\s^.]", " ", text)  # 알파벳, 숫자, 공백을 제외한 문자 제거
    return text

# 텍스트 전처리 적용
data['NEWS_CNTS'] = data['NEWS_CNTS'].apply(clean_text)
print("Text after cleaning:")

/home/lhch9550/anaconda3/lib/python3.8/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/lhch9550/anaconda3/lib/python3.8/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Data loaded!
Text after cleaning:


In [2]:
# 기존의 소분류 라벨
small_label = ['국회_정당', '북한', '선거', '외교', '청와대', '행정_자치', '골프', '농구_배구', '야구_메이저리그',
               '야구_일본프로야구', '올림픽_아시안게임', '축구_월드컵', '축구_한국프로축구', '축구_해외축구',
               '교육_시험', '날씨', '노동_복지', '미디어', '사건_사고', '여성', '의료_건강', '장애인', '환경',
               '미술_건축', '방송_연예', '생활', '요리_여행', '음악', '전시_공연', '종교', '출판', '학술_문화재',
               '러시아', '미국_북미', '아시아', '유럽_EU', '일본', '중국', '중남미', '중동_아프리카', '국제경제',
               '금융_재테크', '무역', '반도체', '부동산', '산업_기업', '서비스_쇼핑', '외환', '유통', '자동차',
               '자원', '증권_증시', '취업_창업', '과학', '모바일', '보안', '인터넷_SNS', '콘텐츠']

# 소분류 라벨에 맞춘 label2id, id2label 생성
small_label2id = {label: i for i, label in enumerate(small_label)}
small_id2label = {i: label for label, i in small_label2id.items()}


In [3]:
# 라벨 변환 함수 정의
def label_to_id(label):
    """쉼표로 구분된 첫 번째 라벨만 선택하고, 해당 라벨을 label2id로 변환"""
    first_label = label.split(',')[0].strip()  # 쉼표로 구분된 첫 번째 라벨만 선택
    
    if first_label in small_label2id:
        return small_label2id[first_label]  # 해당 라벨을 ID로 변환
    else:
        #print(f"Label '{first_label}' not found in label2id.")  # 디버깅 메시지
        return None  # 라벨이 없으면 None 반환

# 'NEWS_SML_SUBJ_CD' 컬럼을 기준으로 라벨을 숫자 ID로 변환
data['label'] = data['NEWS_SML_SUBJ_CD'].apply(label_to_id)

# NaN 값이 있는 행을 드랍
data = data.dropna(subset=['label'])
data

,Unnamed: 0,file_id,doc_id,title,author,publisher,date,topic,original_topic,sentence_id,sentence,NEWS_SML_SUBJ_CD,NEWS_CNTS,label_list,label
0,0,NIRW2300000001,NIRW2300000001.10,노컷뉴스 2022년 기사,경남CBS 최호영 기자 최호영,노컷뉴스,20220101,사회,경제>취업_창업,NIRW2300000001.10.1,경남 12월에만 5698명 ‘역대 최다’…29일 연속 세 자릿수 확산,취업_창업,경남 12월에만 5698명 역대 최다 29일 연속 세 자릿수 확산,['취업_창업'],52.0
1,1,NIRW2300000001,NIRW2300000001.10,노컷뉴스 2022년 기사,경남CBS 최호영 기자 최호영,노컷뉴스,20220101,사회,경제>취업_창업,NIRW2300000001.10.2,지난해 12월 경남에서 발생한 코로나19 확진자는 역대 가장 많은 5700명에 달했다.,취업_창업,지난해 12월 경남에서 발생한 코로나19 확진자는 역대 가장 많은 5700명에 달했다.,['취업_창업'],52.0
2,2,NIRW2300000001,NIRW2300000001.10,노컷뉴스 2022년 기사,경남CBS 최호영 기자 최호영,노컷뉴스,20220101,사회,경제>취업_창업,NIRW2300000001.10.3,경남은 1일 오전 10시 기준으로 도내 3개 시에서 21명의 확진자가 발생했다. 창...,취업_창업,경남은 1일 오전 10시 기준으로 도내 3개 시에서 21명의 확진자가 발생했다. 창...,['취업_창업'],52.0
3,3,NIRW2300000001,NIRW2300000001.10,노컷뉴스 2022년 기사,경남CBS 최호영 기자 최호영,노컷뉴스,20220101,사회,경제>취업_창업,NIRW2300000001.10.4,"이 중 38%인 8명은 도내 또는 다른 지역 확진자의 접촉자, 8명(38%)은 감염...",취업_창업,이 중 38 인 8명은 도내 또는 다른 지역 확진자의 접촉자 8명 38 은 감염...,['취업_창업'],52.0
4,4,NIRW2300000001,NIRW2300000001.10,노컷뉴스 2022년 기사,경남CBS 최호영 기자 최호영,노컷뉴스,20220101,사회,경제>취업_창업,NIRW2300000001.10.5,"기존 집단감염 사례를 보면, 거제 소재 종교시설 관련 확진자는 4명이 추가돼 25명...",취업_창업,기존 집단감염 사례를 보면 거제 소재 종교시설 관련 확진자는 4명이 추가돼 25명...,['취업_창업'],52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734489,734489,NPRW2300000001,NPRW2300000001.31892,머니투데이 2022년 기사,대전=허재구|기자|,머니투데이,20220628,IT/과학,IT_과학>모바일,NPRW2300000001.31892.7,"나아가, 특허청은 올해부터 WIPO와 협의해 고객지원 전문가의 역할을 확대하고 국내...",모바일,나아가 특허청은 올해부터 WIPO와 협의해 고객지원 전문가의 역할을 확대하고 국내...,['모바일'],54.0
734490,734490,NPRW2300000001,NPRW2300000001.31892,머니투데이 2022년 기사,대전=허재구|기자|,머니투데이,20220628,IT/과학,IT_과학>모바일,NPRW2300000001.31892.8,"앞으로 고객지원 전문가는 출원인, 특허사무소 대리인과 긴밀한 소통을 통해 ePCT ...",모바일,앞으로 고객지원 전문가는 출원인 특허사무소 대리인과 긴밀한 소통을 통해 ePCT ...,['모바일'],54.0
734491,734491,NPRW2300000001,NPRW2300000001.31892,머니투데이 2022년 기사,대전=허재구|기자|,머니투데이,20220628,IT/과학,IT_과학>모바일,NPRW2300000001.31892.9,"이처럼 WIPO가 한국에 고객지원 전문가를 배치하고, 사용자 지원을 더욱 강화하기로...",모바일,이처럼 WIPO가 한국에 고객지원 전문가를 배치하고 사용자 지원을 더욱 강화하기로...,['모바일'],54.0
734492,734492,NPRW2300000001,NPRW2300000001.31892,머니투데이 2022년 기사,대전=허재구|기자|,머니투데이,20220628,IT/과학,IT_과학>모바일,NPRW2300000001.31892.10,실제로 지난해 우리나라의 PCT 국제특허 출원은 전년 동기 대비 3.2% 증가한 2...,모바일,실제로 지난해 우리나라의 PCT 국제특허 출원은 전년 동기 대비 3.2 증가한 2...,['모바일'],54.0


In [4]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# 데이터 샘플링 함수
def sampling_func(data, sample_pct, _seed):
    np.random.seed(_seed)
    N = len(data)  # 데이터 길이
    sample_n = int(N * sample_pct)  # 샘플링할 데이터 수
    # DataFrame에서 무작위로 샘플링
    sample = data.sample(n=sample_n, random_state=_seed)  # pandas sample 사용
    return sample

# ClsDataset 클래스 정의
class ClsDataset(Dataset):
    def __init__(self, data):
        self.data = data  # 샘플링된 데이터
        # 데이터에서 'NEWS_CNTS'와 'label'을 분리
        self.texts = data['NEWS_CNTS'].tolist()  # DataFrame에서 텍스트 컬럼 가져오기
        self.labels = data['label'].tolist()  # DataFrame에서 라벨 컬럼 가져오기

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # 텍스트와 라벨을 반환
        text = self.texts[idx]
        label = self.labels[idx]
        return text, label

# 수정된 load_dataset 함수
def load_dataset(data, con):
    # 결측값 처리: 'label' 컬럼에 NaN 값이 있는 행은 제거
    data = data.dropna(subset=['label'])
    
    # 'label' 컬럼에 NaN 처리 및 정수 값 변환
    data['label'] = data['label'].apply(lambda x: int(x) if isinstance(x, (int, float)) else 0)
    
    # num_labels로 설정한 범위 내로 라벨 값 제한 (예: 0에서 num_labels-1 사이)
    data['label'] = data['label'].apply(lambda x: max(0, min(x, con.num_labels - 1)))

    # 샘플링 비율 조정: train, valid, test 분할
    train_pct = 0.7  # train 비율
    valid_pct = 0.2  # valid 비율
    test_pct = 0.1   # test 비율

    # 데이터에서 'train', 'valid', 'test'로 분할
    train_data = data.groupby('label', group_keys=False).apply(sampling_func, sample_pct=train_pct, _seed=con.seed)
    train_data.sort_index()

    # valid 데이터는 train 데이터를 제외한 나머지에서 샘플링
    last_data = data.loc[~data.index.isin(train_data.index)].reset_index(drop=True)
    valid_data = last_data.groupby('label', group_keys=False).apply(sampling_func, sample_pct=valid_pct/(valid_pct + test_pct), _seed=con.seed)

    # test 데이터는 valid 데이터를 제외한 나머지에서 추출
    test_data = last_data.loc[~last_data.index.isin(valid_data.index)].reset_index(drop=True)

    # 각 데이터셋 크기 출력 (디버깅용)
    print(f"Train data size: {len(train_data)}")
    print(f"Valid data size: {len(valid_data)}")
    print(f"Test data size: {len(test_data)}")

    # DataLoader 준비
    train_dataset = ClsDataset(train_data)
    train_dataloader = DataLoader(
        dataset=train_dataset,
        batch_size=con.batch_size,
        shuffle=True,
    )
    
    valid_dataset = ClsDataset(valid_data)
    valid_dataloader = DataLoader(
        dataset=valid_dataset,
        batch_size=con.batch_size,
        shuffle=True,
    )
    
    test_dataset = ClsDataset(test_data)
    test_dataloader = DataLoader(
        dataset=test_dataset,
        batch_size=con.batch_size,
        shuffle=False,
    )
    
    return train_dataloader, valid_dataloader, test_dataloader

In [5]:
from torch.utils.data import DataLoader
import torch
from transformers import BertConfig, BertForSequenceClassification, AutoTokenizer, AdamW
import torch.nn.functional as F
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm

# Config 클래스를 통해 설정값을 가져옵니다.
from config import Config
_config = Config()
_config.num_labels = len(data['label'].unique())  # 라벨의 고유 개수를 설정

# 모델을 GPU로 이동
def initialize_model(con, label2id):
    model = BertForSequenceClassification.from_pretrained(con.model_name, num_labels=len(small_label2id),ignore_mismatched_sizes=True)
    model.to(con.device)  # 모델을 GPU로 이동
    tokenizer = AutoTokenizer.from_pretrained(con.model_name)
    return model, tokenizer

# 옵티마이저 설정
def set_optimizer(model, con):
    optimizer_grouped_parameters = [
        {'params': model.bert.parameters(), 'lr': 3e-5},
        {'params': model.classifier.parameters(), 'lr': con.learning_rate}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=con.learning_rate, eps=con.adam_epsilon, no_deprecation_warning=True)
    return optimizer

# 학습, 평가, 테스트 에포크 함수
from tqdm import tqdm

# train_epoch에 tqdm 적용
def train_epoch(epoch, model, dataloader, optimizer, tokenizer, con):
    model.train()
    total_loss = 0.0
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1} Training", unit="batch")  

    for batch in progress_bar:
        model.zero_grad()
        
        text_inputs = [str(text) if isinstance(text, (str, int, float)) else "" for text in batch[0]]

        sample = tokenizer(
            text_inputs, 
            padding='max_length', 
            truncation=True, 
            max_length=con.max_seq_len, 
            return_tensors="pt", 
            return_token_type_ids=False, 
            return_attention_mask=True,  # 
            return_offsets_mapping=False
        )['input_ids']

        _label = batch[1].to(con.device)
        samples = sample.to(con.device)
        labels = torch.tensor(_label).to(con.device)  #

        outputs = model(samples, labels=labels)
        loss = outputs.loss  # 

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), con.max_grad_norm)
        optimizer.step()
        total_loss += loss.item()

        progress_bar.set_postfix(loss=loss.item())

    return total_loss / len(dataloader)


def valid_epoch(epoch, dataloader, model, tokenizer, con, id2label):
    total_loss = 0.0
    total_correct = 0
    total_len = 0
    model.eval()
    all_token_predictions = []
    all_token_labels = []
    
    tepoch = tqdm(dataloader, unit="batch", leave=False)
    for batch in tepoch:
        tepoch.set_description(f"Valid")
        with torch.no_grad():
            sample = tokenizer(batch[0], padding='max_length', truncation=True, stride=con.stride,
                               max_length=con.max_seq_len, return_tensors="pt")['input_ids']
            _label = batch[1].to(con.device)
            samples = sample.to(con.device)
            labels = torch.tensor(_label).to(con.device)

            outputs = model(samples, labels=labels)
            loss, logits = outputs[:2]
            total_loss += loss.item()

            pred = torch.argmax(F.softmax(logits), dim=1)
            correct = pred.eq(labels)
            total_correct += correct.sum().item()
            total_len += len(labels)
            all_token_labels.extend(labels.cpu().numpy())
            all_token_predictions.extend(pred.cpu().numpy())

        tepoch.set_postfix(loss=loss.mean().item())
    
    # F1 Score 계산
    all_token_labels = [id2label[int(x)] for x in all_token_labels]
    all_token_predictions = [id2label[int(x)] for x in all_token_predictions]
    token_f1 = f1_score(all_token_labels, all_token_predictions, average="micro")
    print('[Epoch {}] -> F1_score: {:.4f}, Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, token_f1, total_loss / len(dataloader), total_correct / total_len))
    
    return total_loss / len(dataloader), token_f1

def test_epoch(dataloader, model, tokenizer, con, id2label):
    total_loss = 0.0
    total_correct = 0
    total_len = 0
    model.eval()
    all_token_predictions = []
    all_token_labels = []
    
    tepoch = tqdm(dataloader, unit="batch", leave=False)
    for batch in tepoch:
        tepoch.set_description(f"Test")
        with torch.no_grad():
            sample = tokenizer(batch[0], padding='max_length', truncation=True, stride=con.stride,
                               max_length=con.max_seq_len, return_tensors="pt")['input_ids']
            _label = batch[1].to(con.device)
            samples = sample.to(con.device)
            labels = torch.tensor(_label).to(con.device)
            
            outputs = model(samples, labels=labels)
            loss, logits = outputs[:2]
            total_loss += loss.item()

            pred = torch.argmax(F.softmax(logits), dim=1)
            correct = pred.eq(labels)
            total_correct += correct.sum().item()
            total_len += len(labels)
            all_token_labels.extend(labels.cpu().numpy())
            all_token_predictions.extend(pred.cpu().numpy())

        tepoch.set_postfix(loss=loss.mean().item())
    
    # F1 Score 계산 및 보고서 출력
    all_token_labels = [id2label[int(x)] for x in all_token_labels]
    all_token_predictions = [id2label[int(x)] for x in all_token_predictions]
    token_result = classification_report(all_token_labels, all_token_predictions)
    token_f1 = f1_score(all_token_labels, all_token_predictions, average="micro")

    print(token_result)
    tepoch.set_postfix(loss=total_loss / len(dataloader), token_f1=token_f1)

    return total_loss / len(dataloader), token_f1

# 학습 과정
def train(model, tokenizer, train_dataloader, valid_dataloader, test_dataloader, optimizer, con, small_id2label):
    for epoch in range(con.epoch):
        print(f"Epoch {epoch+1}/{con.epoch}")
        
        # 학습
        train_loss = train_epoch(epoch, model, train_dataloader, optimizer, tokenizer, con)

        # 검증
        valid_loss, valid_f1 = valid_epoch(epoch, valid_dataloader, model, tokenizer, con, small_id2label)

        # 테스트
        test_loss, test_f1 = test_epoch(test_dataloader, model, tokenizer, con, small_id2label)

        print(f"Epoch {epoch+1} Summary:")
        print(f"Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}, Test Loss: {test_loss:.4f}")
        print(f"Valid F1: {valid_f1:.4f}, Test F1: {test_f1:.4f}\n")

In [6]:
# 모델 초기화 및 DataLoader 설정
model, tokenizer = initialize_model(_config, small_label2id)
optimizer = set_optimizer(model, _config)

# 데이터셋 준비
train_dataloader, valid_dataloader, test_dataloader = load_dataset(data, _config)

/home/lhch9550/anaconda3/lib/python3.8/site-packages/transformers/modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map

Train data size: 341343
Valid data size: 97533
Test data size: 48786


In [ ]:
# 학습 호출
train(model, tokenizer, train_dataloader, valid_dataloader, test_dataloader, optimizer, _config, small_id2label)

In [26]:
def save_model(model, tokenizer, save_path):
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    print(f"Model saved at: {save_path}")

# 모델 저장 경로
save_path = "/home/lhch9550/공모전/KPF-BERT-CLS/saved_model"

# 모델 저장 실행
save_model(model, tokenizer, save_path)

✅ Model saved at: /home/lhch9550/공모전/KPF-BERT-CLS/saved_model


In [11]:
import random

def predict(model, tokenizer, test_dataloader, con, small_id2label):
    model.eval()  # 평가 모드
    predictions = []
    true_labels = []
    texts = []

    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Predicting"):
            text_inputs = [str(text) for text in batch[0]]  # 입력 텍스트
            encoded_inputs = tokenizer(text_inputs, padding=True, truncation=True, max_length=con.max_seq_len, return_tensors="pt")
            encoded_inputs = {k: v.to(con.device) for k, v in encoded_inputs.items()}
            
            outputs = model(**encoded_inputs)
            logits = outputs.logits  # 예측값
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            
            predictions.extend(preds)
            true_labels.extend(batch[1].cpu().numpy())  # 실제 정답
            texts.extend(batch[0])  # 원본 텍스트 저장

    # 라벨 ID → 라벨명 변환
    pred_labels = [small_id2label[p] for p in predictions]
    true_labels = [small_id2label[t] for t in true_labels]

    # 결과 데이터프레임 생성
    df_results = pd.DataFrame({"Text": texts, "True Label": true_labels, "Predicted Label": pred_labels})

    # 버깅: 샘플 5개 확인 (랜덤)
    sample_indices = random.sample(range(len(df_results)), 5)
    print("\n🔍 샘플 데이터 정렬 검증")
    for idx in sample_indices:
        print(f"\n📝 Text: {df_results.iloc[idx]['Text']}")
        print(f"True Label: {df_results.iloc[idx]['True Label']}")
        print(f"Predicted Label: {df_results.iloc[idx]['Predicted Label']}")

    # 성능 평가 지표 출력
    print("\n[Classification Report]\n")
    print(classification_report(true_labels, pred_labels, digits=4))

    return df_results

# 모델과 토크나이저 불러오기
saved_model_path = "/home/lhch9550/공모전/KPF-BERT-CLS/saved_model"
model = BertForSequenceClassification.from_pretrained(saved_model_path).to(_config.device)
tokenizer = AutoTokenizer.from_pretrained(saved_model_path)

# 추론 실행
df_results = predict(model, tokenizer, test_dataloader, _config, small_id2label)

Predicting: 100%|██████████| 11872/11872 [02:33<00:00, 77.19it/s]



🔍 샘플 데이터 정렬 검증

📝 Text: 오 시장은  우선 주택공급 정상화를 위한 제도적 기반을 완비하고 신속통합기획을 적용한 재개발 재건축 사업을 조속히 추진하겠다 며  노후 저층 주거지역을 묶는 소규모 주택정비사업인 모아주택  모아타운도 더욱 활성화하겠다 고 말했다.
✅ True Label: 부동산
🔮 Predicted Label: 러시아

📝 Text: 러시아 언론 스포츠 익스프레스에 따르면 팬들이  범인 으로 특정한 선수는 다름 아닌 러시아올림픽위원회 ROC 의 안드레이 모잘레프다.
✅ True Label: 올림픽_아시안게임
🔮 Predicted Label: 러시아

📝 Text: 옐런 장관은 현재 미 재무장관직을 수행중이지만  미 연준 이사와 샌프란시스코 연방은행 총재  연준 이사회 부의장을 거쳐 연준 이사회 의장을 지낸 이력을 가지고 있다.
✅ True Label: 외교
🔮 Predicted Label: 외교

📝 Text: 경남 거창군은 13일부터 22일까지 열흘 간 거창창포원 일원에서  제3회 아리미아 꽃 축제 를 개최한다고 밝혔다.
✅ True Label: 요리_여행
🔮 Predicted Label: 전시_공연

📝 Text: LG엔솔  내일 코스피 상장  따상  성공할까
✅ True Label: 증권_증시
🔮 Predicted Label: 러시아

[Classification Report]



/home/lhch9550/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lhch9550/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          골프     0.7356    0.8192    0.7752      1311
       교육_시험     0.7490    0.5957    0.6636      4323
        국제경제     0.0000    0.0000    0.0000      2471
       국회_정당     0.5784    0.6919    0.6301     11735
      금융_재테크     0.0000    0.0000    0.0000      4892
          날씨     0.8525    0.7553    0.8010       421
       노동_복지     0.5797    0.6101    0.5945      2670
       농구_배구     0.6928    0.7969    0.7412      2708
         러시아     0.0325    0.9168    0.0627      2487
          무역     0.0000    0.0000    0.0000      2390
       미국_북미     0.0000    0.0000    0.0000      1789
         미디어     0.4938    0.0999    0.1661       801
       미술_건축     0.5904    0.2055    0.3048       842
         반도체     0.0000    0.0000    0.0000      1997
       방송_연예     0.7114    0.6856    0.6983      4729
         부동산     0.0000    0.0000    0.0000      3636
          북한     0.7328    0.6363    0.6811      2422
       사건_사고     0.7512    

/home/lhch9550/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
